In [2]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
drive='/content/drive/MyDrive/Alex_weekly_reports'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv(drive+'/merged.csv')

In [4]:
pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 5.8 MB/s eta 0:00:00


In [5]:
##Tokenize each row of the column
#Run these tokenized rows through doc2Vec model
#utilize these vectors and run through calssfication and see which model gives the most accurate score
import nltk
import regex as re
import string
import emoji
nltk.download('punkt')
from tqdm import tqdm
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
#Function to Remove Puncutation
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ')
    return text

In [7]:
#Data Cleaning and Combining of the Columns
df['mainContent'] = df['mainContent'].fillna('')
#df['title'] = df['title'].apply(remove_punctuations)
#df['title'] = df['title'].str.replace('[^a-zA-Z0-9\s]', '')
df['mainContent'] = df['mainContent'].apply(remove_punctuations)
df['mainContent'] = df['mainContent'].apply(lambda s: emoji.replace_emoji(s, ''))
df['mainContent'] = df['mainContent'].str.replace('[^a-zA-Z\s]', '')

df['combinedContent'] = df['mainContent'] #+ df['title']
print(df['combinedContent'][0])

Opera for Android is a fast all around browser for users who value privacy  customization  and sleek design  Browse the web with security features  such as a built in VPN and ad blocker  while enjoying the easy to use interface  VPN  The first browser  which offers free  built in VPN   Ad blocker  Get rid of thousands of intrusive ads and speed up your browsing Private browsing  Easily switch between normal and private tabs to surf the web without leaving a trace Password Manager  Opt in for automatic saving of passwords and safely autofill your payment information for secure and private online shopping Night Mode  Fully configurable mode with adjustable dimming  color temperature  as well as dark theme and dark web pages choice Reader Mode  Regardless of a websites design  it displays articles in an optimised format that improves their readability  Sync  Simplified synchronization between devices lets access speed dials  open tabs  as well as keep a shared set of bookmarks  typed hist

<ipython-input-7-5ce2b8dea748>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['mainContent'] = df['mainContent'].str.replace('[^a-zA-Z\s]', '')


In [8]:
#Tokenization of the Column
import nltk
from nltk.corpus import stopwords

def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 3:
                continue
            tokens.append(word.lower())
    return tokens

tagged = df.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['combinedContent']), tags=[r.title]), axis=1)
len(tagged)
for i in tagged:
  print(i)

TaggedDocument<['opera', 'for', 'android', 'fast', 'all', 'around', 'browser', 'for', 'users', 'who', 'value', 'privacy', 'customization', 'and', 'sleek', 'design', 'browse', 'the', 'web', 'with', 'security', 'features', 'such', 'built', 'vpn', 'and', 'blocker', 'while', 'enjoying', 'the', 'easy', 'use', 'interface', 'vpn', 'the', 'first', 'browser', 'which', 'offers', 'free', 'built', 'vpn', 'blocker', 'get', 'rid', 'thousands', 'intrusive', 'ads', 'and', 'speed', 'your', 'browsing', 'private', 'browsing', 'easily', 'switch', 'between', 'normal', 'and', 'private', 'tabs', 'surf', 'the', 'web', 'without', 'leaving', 'trace', 'password', 'manager', 'opt', 'for', 'automatic', 'saving', 'passwords', 'and', 'safely', 'autofill', 'your', 'payment', 'information', 'for', 'secure', 'and', 'private', 'online', 'shopping', 'night', 'mode', 'fully', 'configurable', 'mode', 'with', 'adjustable', 'dimming', 'color', 'temperature', 'well', 'dark', 'theme', 'and', 'dark', 'web', 'pages', 'choice', '

In [9]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [10]:
#Applying Doc2Vec
model_dbow = Doc2Vec(dm=0, vector_size= 50, window=3, hs=0, min_count=2, sample = 0, negative=5,workers=cores,epochs=60)
Tagged_docsee=[x for x in tqdm(tagged.values)]
model_dbow.build_vocab(Tagged_docsee)
model_dbow.train(Tagged_docsee, total_examples=model_dbow.corpus_count, epochs=model_dbow.epochs)

100%|██████████| 227/227 [00:00<00:00, 77805.59it/s]


In [11]:
#Getting the Vectors and Splitting the Data
from sklearn.model_selection import train_test_split
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    regressor = list(model.infer_vector(doc.words) for doc in sents)
    return regressor

vectors = vec_for_learning(model_dbow, tagged)
print(vectors[222])
print(len(vectors))
labels = df['isVault']
print(labels[222])
print(len(labels))
X_train, X_test, y_train, y_test = train_test_split(vectors, labels, test_size = 0.4, random_state=40)

[-0.74235725  0.1707145  -0.12396093  0.80522627  0.35293254 -0.32915303
 -0.18642758  2.753042   -0.09224159 -0.5821583   1.367068    0.862253
  0.710798   -0.60045165 -1.5573101  -0.604243    1.0629718   1.2271369
 -0.658869   -0.4333256  -1.2406236   0.3868074  -2.6910079  -0.97260374
  0.50359666 -1.1091397  -0.9221456  -0.9823745   0.33150357 -0.6846896
  1.8208082   1.5085577  -0.6897799   0.9428457  -1.0945569   0.4583271
  0.0156048  -1.2989215  -0.38155594 -0.53755647 -0.5813302   0.2895631
  0.41447997 -0.21632715 -0.7969821   0.14432967  0.82072246  0.6845193
  0.9414938  -1.2754924 ]
227
1
227


In [12]:
#for i in range(213):
  #print(len(vectors[i]))

In [13]:
len(vectors[199])

50

In [14]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

print(y_test)
print(y_pred)

#Logistic Regression Accuracy Score
from sklearn import metrics
print('Accuracy Score:')
logreg = metrics.accuracy_score(y_test, y_pred)
print(logreg)
print(metrics.classification_report( y_test,y_pred, labels=None))

69     0
112    0
200    0
173    0
192    0
      ..
0      0
53     0
48     0
142    1
197    0
Name: isVault, Length: 91, dtype: int64
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 1 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 1
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
Accuracy Score:
0.8901098901098901
              precision    recall  f1-score   support

           0       0.88      0.98      0.93        66
           1       0.94      0.64      0.76        25

    accuracy                           0.89        91
   macro avg       0.91      0.81      0.85        91
weighted avg       0.90      0.89      0.88        91



In [15]:
#SVM
from sklearn import svm

svc = svm.SVC(kernel = 'linear')

svc.fit(X_train, y_train)

y_pred = svc.predict(X_test)

print(y_test)
print(y_pred)

#SVM Accuracy Score
from sklearn import metrics
print('Accuracy Score:')
svm = metrics.accuracy_score(y_test, y_pred)
print(svm)
print(metrics.classification_report( y_test,y_pred, labels=None))

69     0
112    0
200    0
173    0
192    0
      ..
0      0
53     0
48     0
142    1
197    0
Name: isVault, Length: 91, dtype: int64
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 1 0 0
 1 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0 1 1 1 0 0 0 1 0 1 0 1 1 0 1 0 1
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
Accuracy Score:
0.8461538461538461
              precision    recall  f1-score   support

           0       0.88      0.91      0.90        66
           1       0.74      0.68      0.71        25

    accuracy                           0.85        91
   macro avg       0.81      0.79      0.80        91
weighted avg       0.84      0.85      0.84        91



In [16]:
#Naive Bayes
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()

gnb.fit(X_train, y_train)

y_pred = gnb.predict(X_test)

print(y_test)
print(y_pred)

#Naive Bayes Accuracy Score
from sklearn import metrics
print('Accuracy Score:')
nbayes = metrics.accuracy_score(y_test, y_pred)
print(nbayes)
print(metrics.classification_report( y_test,y_pred, labels=None))

69     0
112    0
200    0
173    0
192    0
      ..
0      0
53     0
48     0
142    1
197    0
Name: isVault, Length: 91, dtype: int64
[0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1
 1 1 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0]
Accuracy Score:
0.8021978021978022
              precision    recall  f1-score   support

           0       0.82      0.94      0.87        66
           1       0.73      0.44      0.55        25

    accuracy                           0.80        91
   macro avg       0.77      0.69      0.71        91
weighted avg       0.79      0.80      0.78        91



In [17]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

print(y_test)
print(y_pred)

#Random Forest Accuracy Score
from sklearn import metrics
print('Accuracy Score:')
rforest = metrics.accuracy_score(y_test, y_pred)
print(rforest)
print(metrics.classification_report( y_test,y_pred, labels=None))

69     0
112    0
200    0
173    0
192    0
      ..
0      0
53     0
48     0
142    1
197    0
Name: isVault, Length: 91, dtype: int64
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
Accuracy Score:
0.7692307692307693
              precision    recall  f1-score   support

           0       0.76      0.98      0.86        66
           1       0.83      0.20      0.32        25

    accuracy                           0.77        91
   macro avg       0.80      0.59      0.59        91
weighted avg       0.78      0.77      0.71        91



In [18]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()

dt.fit(X_train, y_train)

y_pred = dt.predict(X_test)

print(y_test)
print(y_pred)

#Decision Tree Accuracy Score
from sklearn import metrics
print('Accuracy Score:')
dtree = metrics.accuracy_score(y_test, y_pred)
print(dtree)
print(metrics.classification_report( y_test,y_pred, labels=None))

69     0
112    0
200    0
173    0
192    0
      ..
0      0
53     0
48     0
142    1
197    0
Name: isVault, Length: 91, dtype: int64
[1 0 0 1 0 0 0 0 0 1 1 0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 1
 1 1 0 0 0 1 1 1 1 1 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 1 1 1 0 1 0 1
 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
Accuracy Score:
0.7032967032967034
              precision    recall  f1-score   support

           0       0.83      0.74      0.78        66
           1       0.47      0.60      0.53        25

    accuracy                           0.70        91
   macro avg       0.65      0.67      0.66        91
weighted avg       0.73      0.70      0.71        91



In [19]:
#K-Nearest Neighbor
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

print(y_test)
print(y_pred)

#K-Nearest Neighbor Accuracy Score
from sklearn import metrics
print('Accuracy Score:')
knn = metrics.accuracy_score(y_test, y_pred)
print(knn)

69     0
112    0
200    0
173    0
192    0
      ..
0      0
53     0
48     0
142    1
197    0
Name: isVault, Length: 91, dtype: int64
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
Accuracy Score:
0.7692307692307693


In [20]:
index = ['Text Embedding Classification']
table = pd.DataFrame({'Logistic Regression': round(logreg * 100, 2), 'Support Vector Machine': round(svm * 100, 2), 'Naive Bayes': round(nbayes * 100, 2), 'Random Forest': round(rforest * 100, 2), 'Decision Tree': round(dtree * 100, 2), 'K-Nearest Neighbor': round(knn * 100, 2)}, index = index)
print(table)

                               Logistic Regression  Support Vector Machine  \
Text Embedding Classification                89.01                   84.62   

                               Naive Bayes  Random Forest  Decision Tree  \
Text Embedding Classification        80.22          76.92          70.33   

                               K-Nearest Neighbor  
Text Embedding Classification               76.92  


In [21]:
print(round(logreg * 100, 2))
print(round(svm * 100, 2))
print(round(nbayes * 100, 2))
print(round(rforest * 100, 2))
print(round(knn * 100, 2))
print(round(dtree * 100, 2))

89.01
84.62
80.22
76.92
76.92
70.33
